### Manual test queries

Notebook to manually inspect the DuckDB tables without needing a full SQL client

In [1]:
import duckdb
db_path = '/workspaces/wikipedia-api-to-duckdb/wiki_recent_changes.db'


In [5]:
# Execute a query to get the list of tables
# This query selects all table names from the "tables" system table in the "main" schema

with duckdb.connect(db_path) as conn:
    query = "SELECT table_name FROM information_schema.tables WHERE table_schema='main';"
    tables = conn.execute(query).fetchall()

    # Print the list of tables
    print("Tables:")
    for table in tables:
        print("     "+table[0])



Tables:
     bronze_recent_changes
     silver_recent_changes
     gold_most_active_timeslots
     gold_timeslot_highest_activity


In [ ]:
#show 1 row
with duckdb.connect(db_path) as conn:
    result = conn.execute('SELECT * FROM bronze_recent_changes LIMIT 1').fetchdf()
print(result)


  actionhidden  anon bot                                            comment  \
0         None  None      Move 1 url. [[User:GreenC/WaybackMedic_2.5|Way...   

  commenthidden logaction logid logparams logtype minor  ...  \
0          None      None  None      None    None  None  ...   

                                       sha1 sha1hidden suppressed tags  \
0  7f8fcf1f49a1d1ae2c637e9ec8180d6d96765519       None       None   []   

              timestamp      title  type        user userhidden    userid  
0  2024-10-31T23:59:59Z  Brad Pitt  edit  GreenC bot       None  27823944  

[1 rows x 30 columns]


In [ ]:
#count entries in base table
with duckdb.connect(db_path) as conn:
    result = conn.execute('SELECT count(*) FROM bronze_recent_changes').fetchdf()
print(result)


   count_star()
0        356417


In [ ]:
# Use PRAGMA table_info to get the columns of the bronze table
with duckdb.connect(db_path) as conn: 
    columns_info = conn.execute("PRAGMA table_info('bronze_recent_changes')").fetchdf()
print(columns_info)

    cid           name     type  notnull dflt_value     pk
0     0   actionhidden  VARCHAR    False       None  False
1     1           anon  VARCHAR    False       None  False
2     2            bot  VARCHAR    False       None  False
3     3        comment  VARCHAR    False       None  False
4     4  commenthidden  VARCHAR    False       None  False
5     5      logaction  VARCHAR    False       None  False
6     6          logid  VARCHAR    False       None  False
7     7      logparams  VARCHAR    False       None  False
8     8        logtype  VARCHAR    False       None  False
9     9          minor  VARCHAR    False       None  False
10   10            new  VARCHAR    False       None  False
11   11         newlen  VARCHAR    False       None  False
12   12             ns  VARCHAR    False       None  False
13   13      old_revid  VARCHAR    False       None  False
14   14         oldlen  VARCHAR    False       None  False
15   15         pageid  VARCHAR    False       None  Fal

In [ ]:
# Use PRAGMA table_info to get the columns of the bronze table
with duckdb.connect(db_path) as conn: 
    columns_info = conn.execute("PRAGMA table_info('silver_recent_changes')").fetchdf()
print(columns_info)

    cid           name       type  notnull dflt_value     pk
0     0   actionhidden    BOOLEAN    False       None  False
1     1           anon    BOOLEAN    False       None  False
2     2            bot    BOOLEAN    False       None  False
3     3        comment    VARCHAR    False       None  False
4     4  commenthidden    BOOLEAN    False       None  False
5     5      logaction    VARCHAR    False       None  False
6     6          logid    INTEGER    False       None  False
7     7      logparams    VARCHAR    False       None  False
8     8        logtype    VARCHAR    False       None  False
9     9          minor    BOOLEAN    False       None  False
10   10            new    BOOLEAN    False       None  False
11   11         newlen    INTEGER    False       None  False
12   12             ns    INTEGER    False       None  False
13   13      old_revid    INTEGER    False       None  False
14   14         oldlen    INTEGER    False       None  False
15   15         pageid  

In [ ]:
#check aggregated table
with duckdb.connect(db_path) as conn:
    result = conn.execute('SELECT * FROM gold_most_active_timeslots').fetchdf()
print(result)

    shifted_slot_start    shifted_slot_end  change_count
0  2024-10-31 00:00:00 2024-10-31 00:30:00          4930
1  2024-10-31 00:15:00 2024-10-31 00:45:00          4009
2  2024-10-31 00:30:00 2024-10-31 01:00:00          4978
3  2024-10-31 00:45:00 2024-10-31 01:15:00          3209
4  2024-10-31 01:00:00 2024-10-31 01:30:00          3348
..                 ...                 ...           ...
91 2024-10-31 22:45:00 2024-10-31 23:15:00          6561
92 2024-10-31 23:00:00 2024-10-31 23:30:00          6739
93 2024-10-31 23:15:00 2024-10-31 23:45:00          5241
94 2024-10-31 23:30:00 2024-11-01 00:00:00          4447
95 2024-10-31 23:45:00 2024-11-01 00:15:00          4928

[96 rows x 3 columns]


In [ ]:
#get timeslot with highest activity
with duckdb.connect(db_path) as conn:
    result = conn.execute('''
        SELECT *
        FROM gold_most_active_timeslots
        ORDER BY change_count DESC
        LIMIT 1;
        ''').fetchdf()
print(result)

   shifted_slot_start    shifted_slot_end  change_count
0 2024-10-31 23:00:00 2024-10-31 23:30:00          6739


In [ ]:
#check final answer
with duckdb.connect(db_path) as conn:
    result = conn.execute('SELECT * FROM gold_timeslot_highest_activity').fetchdf()
print(result)



   shifted_slot_start    shifted_slot_end  change_count
0 2024-10-31 23:00:00 2024-10-31 23:30:00          6739
